### Mercari Price 
The files consist of a list of product listings. These files are tab-delimited.

Fields:
- train_id or test_id - the id of the listing

- name - the title of the listing. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid  leakage. These removed prices are represented as [rm]

- item_condition_id - the condition of the items provided by the seller

- category_name - category of the listing

- brand_name

- price - the price that the item was sold for. This is the target variable that you will predict. The unit is USD. This column doesn't exist in test.tsv since that is what you will predict.

- shipping - 1 if shipping fee is paid by seller and 0 by buyer

- item_description - the full description of the item. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid leakage. These removed prices are represented as [rm]

## Setup

In [ ]:
! pip install pydot graphviz emoji transformers

import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
import nltk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow.keras.backend as K


from tqdm._tqdm_notebook import tqdm_notebook

import os
import itertools

import matplotlib.pyplot as plt
import itertools
from collections import Counter
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import Dense, Input, Embedding, Concatenate, Flatten, Dropout, LSTM, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

from nltk.corpus import stopwords
import string

import pickle

from tensorflow.keras.preprocessing.text import text_to_word_sequence 
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import TweetTokenizer

import emoji
import os

from sklearn.feature_extraction.text import CountVectorizer

import transformers
from transformers import DistilBertTokenizer, TFDistilBertModel, pipeline

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


stop_words = set(stopwords.words('english'))
stop_words.remove("no")

tqdm_notebook.pandas()

## Colab

In [ ]:
RunningInCOLAB = 'google.colab' in str(get_ipython())

# check if in colab
if RunningInCOLAB and not os.path.isdir('/content/gdrive'):
    print("Running in colab")
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    colab_root = '/content/drive'
      
if RunningInCOLAB:
    root_dir = "/content/gdrive/My Drive/"
    base_dir = root_dir + 'project-mercari-price/'
    if not os.path.isdir(base_dir):
        os.mkdir(base_dir)
else:
    root_dir= os.getcwd()
    base_dir = root_dir

os.chdir(base_dir)

os.getcwd()

## Dataset download

In [ ]:
dataset_downloaded_path = os.path.join(base_dir, "dataset_downloaded.ignore")
dataset_downloaded = os.path.isfile(dataset_downloaded_path)
dataset_downloaded

if not dataset_downloaded:
  # install kaggle to download dataset
  ! pip install kaggle python-dotenv

# set to True if you want to save kaggle credentials into a .env file
persist_credentials = False

if not dataset_downloaded:
  # create .env file containing KAGGLE_USER and KAGGLE_KEY
    kaggle_env = os.path.join(base_dir, '.env')
    if not os.path.isfile(kaggle_env):
        with open(kaggle_env, 'w') as envfile:
            kaggle_user = input("Insert kaggle username")
            kaggle_key = input("Insert kaggle key; generate one from kaggle account")
        if persist_credentials:
            envfile.write(f"""
            KAGGLE_USERNAME={kaggle_user}
            KAGGLE_KEY={kaggle_key}
            """)

        # set env vars
        os.environ["KAGGLE_USERNAME"] = kaggle_user
        os.environ["KAGGLE_KEY"] = kaggle_key

        del kaggle_user
        del kaggle_key

if not dataset_downloaded:
  # loading env vars if .env file exists
    if os.path.isfile(kaggle_env):
        from dotenv import load_dotenv
        load_dotenv(dotenv_path=kaggle_env)
    print(os.environ.get("KAGGLE_USERNAME"))

if not dataset_downloaded:
    # download and extract dataset
    ! kaggle competitions download -c mercari-price-suggestion-challenge

    # create file so that we know we already downloaded
    with open(dataset_downloaded_path, 'w') as dd_file:
        dataset_downloaded = True
        dd_file.write("")

    print('cwd: ', os.getcwd())
    
    os.listdir()

if not dataset_downloaded:
    ! 7z x train.tsv.7z
    ! 7z x test.tsv.7z

os.listdir()

## Load dataset

In [ ]:
dtypes={
    'name': 'string',
    'item_condition_id': 'int32',
    'category_name': 'string',
    'brand_name': 'string',
    'price': 'float',
    'shipping': 'int32',
    'item_description': 'string'
}
data = pd.read_csv("train.tsv", sep='\t', dtype=dtypes)
data = data.drop(columns=["train_id"])
print(data.dtypes)
print(data.shape)
data

In [ ]:
dtypes={
    'name': 'string',
    'item_condition_id': 'int32',
    'category_name': 'string',
    'brand_name': 'string',
    'price': 'float',
    'shipping': 'int32',
    'item_description': 'string'
}
test = pd.read_csv("test.tsv", sep='\t', dtype=dtypes)
test = test.drop(columns=["test_id"])
print(test.dtypes)
print(test.shape)
test

In [ ]:
for column in data.columns:
    print("number of null value in {} : {}".format(column,data[column].isnull().sum()))

In [ ]:
for column in test.columns:
    print("number of null value in {} : {}".format(column,test[column].isnull().sum()))

In [ ]:
data["category_name"].value_counts()

In [ ]:
data["price"].describe()

# Data cleansing

Handle missing values and wrong prices

https://www.mercari.com/us/help_center/article/69

In [ ]:
len(data[data["price"]<5])

In [ ]:
data=data[data["price"]>=5]

In [ ]:
data = data[data["item_description"].notna()]
data["brand_name"] = data["brand_name"].fillna(value="NA")
data["category_name"] = data["category_name"].fillna(value="NA")
# see warnings -> inplace?
data.shape

In [ ]:
test["brand_name"] = test["brand_name"].fillna(value="NA")
test["category_name"] = test["category_name"].fillna(value="NA")
# see warnings -> inplace?
test.shape

In [ ]:
data

# Preprocessing

In [ ]:
data["item_description"]=data["item_description"].str.lower()
data["name"]=data["name"].str.lower()
data.head()

In [ ]:
test["item_description"]=test["item_description"].str.lower()
test["name"]=test["name"].str.lower()
test.head()

In [ ]:
tweetTokenizer = TweetTokenizer()

def list_to_str(l):
   return ' '.join([str(elem) for elem in l])
   
def textCleanup(df):
  df=df.to_frame(name="str")
  #df["clean"] = df["str"].progress_apply(text_to_word_sequence)   # 20 secondi
  global tweetTokenizer
  df["clean"] = df["str"].progress_apply(tweetTokenizer.tokenize) # 2 minutes but correctly handles emojis

  # punct and stop words
  df["clean"] = df["clean"].progress_apply(lambda sentence:
                                           [w for w in sentence if w
                                              not in string.punctuation
                                              and w not in stop_words]) # 18 s

  lemmatizer = WordNetLemmatizer() 
  
  df["clean"] = df["clean"].progress_apply(lambda sentence : [lemmatizer.lemmatize(word) for word in sentence if word not in stop_words]) # 10 secondi
  df["clean"] = df["clean"].progress_apply(list_to_str) # 6 secondi
  return df["clean"]

In [ ]:
def preprocessData(data):
  print('description clean up')
  data["item_description_clean"] = textCleanup(data["item_description"]) 

  print('name clean up')
  data["name_clean"] = textCleanup(data["name"])
  
  return data

In [ ]:
data = preprocessData(data)
data.head()

In [ ]:
test = preprocessData(test)
test.head()

# Load preprocessed

In [ ]:
# save preprocessed data train
force_overwrite = False
fname = 'train_preprocess.npy'

if force_overwrite or not os.path.isfile(fname):
  print("saving preprocess train data")
  data.to_pickle(fname)
else:
  print("loading existing preprocess train data")
  data = pd.read_pickle(fname)

In [ ]:
# save preprocessed data test
force_overwrite = False
fname = 'test_preprocess.npy'

if force_overwrite or not os.path.isfile(fname):
  print("saving preprocess test data")
  test.to_pickle(fname)
else:
  print("loading existing preprocess test data")
  test = pd.read_pickle(fname)

# Encoding

In [ ]:
# TODO ensure it is correct or use a well tested alternative like sklearn (found problems with dimensions)
class LabelEncoder:
  def __init__(self, unknown = 0, invUnknown = 'unknown'):
    self.leDict = {}
    self.invDict = {}
    self.unknown = unknown
    self.invUnknown = invUnknown

  def fit(self, data):
    vci = pd.value_counts(data).index
    self.leDict = dict(zip(vci, range(1, len(vci)+1)))
    self.invDict = dict(zip(range(1, len(vci)+1), vci))

  def transform1(self, item):
    return self.leDict.get(item, self.unknown)

  def transform(self, data):
    return data.apply(lambda item: self.transform1(item))

  def inverse_transform1(self, item):
    return self.invDict.get(item, self.invUnknown)

  def inverse_transform(self, data):
    return data.apply(lambda item: self.inverse_transform1(item))

In [ ]:
cat_le = LabelEncoder()
cat_le.fit(np.hstack([data["category_name"], test["category_name"]]))

data["category_name_l"] = cat_le.transform(data["category_name"])
test["category_name_l"] = cat_le.transform(test["category_name"])

In [ ]:
brand_le = LabelEncoder()
brand_le.fit(np.hstack([data["brand_name"], test["category_name"]]))

data["brand_name_l"] = brand_le.transform(data["brand_name"])
test["brand_name_l"] = brand_le.transform(test["brand_name"])

In [ ]:
# tokenize with keras; it also does some encoding
def tokenizeData(df, description, name, tokenizer= None, texts= None):
  if tokenizer == None:
    tokenizer = Tokenizer()

    print("fit tokenizer")
    tokenizer.fit_on_texts(texts)
  
  print('tokenize description')
  df["item_description_t"]=tokenizer.texts_to_sequences(df[description])

  print('tokenize name')
  df["name_t"]=tokenizer.texts_to_sequences(df[name])
  return df, tokenizer

## Keras Embedding

In [ ]:
texts = np.hstack([data["item_description"], data["name"]])

data_keras, tokenizer = tokenizeData(data ,"item_description" ,"name", None, texts)

In [ ]:
#data_keras_bk = data_keras
test_keras_bk = test_keras

In [ ]:
data_keras=data_keras[["item_condition_id","shipping","category_name_l","brand_name_l", "item_description_t", "name_t", "price"]]
data_keras

In [ ]:
test_keras, _ = tokenizeData(test ,"item_description", "name", tokenizer)

In [ ]:
test_keras=test[["item_condition_id","shipping","category_name_l","brand_name_l", "item_description_t", "name_t"]]
test_keras

In [ ]:
train_keras, validation_keras = train_test_split(data_keras, test_size=0.2, random_state=1000)

In [ ]:
vocab_size= len(tokenizer.word_index)+1
print(vocab_size)

In [ ]:
train_keras[["item_description_t","name_t"]]

In [ ]:
desc_length_max=75
inputDesc_train_keras = pad_sequences(train_keras["item_description_t"],
                                                  padding='post', maxlen=desc_length_max)

In [ ]:
inputDesc_validation_keras = pad_sequences(validation_keras["item_description_t"],
                                                  padding='post', maxlen=desc_length_max)

In [ ]:
name_length_max=10
inputName_train_keras = pad_sequences(train_keras["name_t"], padding='post', maxlen=name_length_max)

In [ ]:
inputName_validation_keras = pad_sequences(validation_keras["name_t"], padding='post', maxlen=name_length_max)

In [ ]:
y_train_keras = train_keras["price"]
y_train_keras

In [ ]:
y_validation_keras = validation_keras["price"]
y_validation_keras

In [ ]:
inputA_train_keras = train_keras[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
inputA_validation_keras = validation_keras[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
name_vocab_size= vocab_size
desc_vocab_size= vocab_size
def getModelKeras():
    inputA = Input(shape=(4,))
    #Ad = Dense(4, activation='relu')(inputA)
    Ad = inputA
    
    inputName = Input(shape=(name_length_max,))
    Ne = Embedding(input_dim=name_vocab_size, output_dim=8, input_length=name_length_max)(inputName)
    #Ne = Embedding(
    #    num_tokens,
    #    embedding_dim,
    #    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    #    trainable=False,
    #)(inputName)
    Nd = LSTM(12,return_sequences=True)(Ne)
    Nd = Flatten()(Nd)

    #Nd = Nf
    
    inputDesc = Input(shape=(desc_length_max,))
    De = Embedding(input_dim=desc_vocab_size, output_dim=16, input_length=desc_length_max)(inputDesc)
    #De = Embedding(
    #    num_tokens,
    #    embedding_dim,
    #    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    #    trainable=False,
    #)(inputDesc)
    Dd = LSTM(16,return_sequences=True)(De)
    Dd = LSTM(8,return_sequences=True)(Dd)
    Dd = Flatten()(Dd)
    #Dd = Df
    
    concat = Concatenate()([Ad, Nd, Dd])

    x = Dropout(0.1)(concat)    
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(16, activation='relu')(x)
    
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[inputA, inputName, inputDesc], outputs=x)
    
    return model
    

In [ ]:
model_keras = getModelKeras()
model_keras.summary()

In [ ]:
plot_model(model_keras)

In [ ]:
msle = tf.keras.losses.MeanSquaredLogarithmicError()

def root_mean_squared_logarithmic_error(y_true, y_pred):
    return K.sqrt(msle(y_true, y_pred))

In [ ]:
model_keras.compile(loss = root_mean_squared_logarithmic_error, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history_keras = model_keras.fit(x=[inputA_train_keras, inputName_train_keras, inputDesc_train_keras], y=y_train_keras,
                    epochs=50,
                    verbose=True,
                    validation_data=([inputA_validation_keras, inputName_validation_keras, inputDesc_validation_keras], y_validation_keras),
                    callbacks=[callback],
                    batch_size=512)
# senza pulizia  val_root_mean_squared_logarithmic_error: 0.4701 e loss: 0.3637
# con pulizia val_root_mean_squared_logarithmic_error: 0.4670 e loss: 0.3699

In [ ]:
histDf = pd.DataFrame(history.history)
histDf.tail()

In [ ]:
histDf.plot(y=["loss", "val_loss"])

In [ ]:
inputA_test_keras = test_keras[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]].values.astype('int32')
inputName_test_keras = pad_sequences(test_keras["name_t"], padding='post', maxlen=name_length_max)
inputDesc_test_keras = pad_sequences(test_keras["item_description_t"], padding='post', maxlen=desc_length_max)
pred_keras = model_keras.predict([inputA_test_keras, inputName_test_keras, inputDesc_test_keras])
pred_keras

In [ ]:
import math

def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [ ]:
validation_keras

In [ ]:
pred_keras=pred_keras.round()

In [ ]:
pred_v_keras = model.predict([inputA_validation_keras, inputName_validation_keras, inputDesc_validation_keras])
pred_v_keras = pred_v_keras.round()

In [ ]:
len(pred_v_keras)

In [ ]:
len(y_validation_keras)

In [ ]:
root_mean_squared_logarithmic_error(y_validation_keras, pred_v_keras)

## Glove pretrained

In [ ]:
archive_url = 'http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip'
archive_name = 'glove.6B.zip'
remove_archive = False
embedding_fname = 'glove.6B.100d.txt'

In [ ]:
if not os.path.isfile(archive_name) and not os.path.isfile(embedding_fname):
  ! wget {archive_url} -O {archive_name}

In [ ]:
if not os.path.isfile(embedding_fname):
  ! unzip {archive_name} {embedding_fname}

# remove archive if already extracted
if remove_archive and os.path.isfile(embedding_fname):
  os.remove(archive_name)

Needs a word encoding index (e.g. keras one)

In [ ]:
word_index = tokenizer.word_index
print(len(word_index))

In [ ]:
path_to_glove_file = os.path.join(
    base_dir, "glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
#glove pretrained embedding
num_tokens = len(word_index) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

## Count Vectorizer

In [ ]:
data.head()

In [ ]:
vectorizer_desc = CountVectorizer()
vectorizer_desc.fit(data["item_description_clean"].values)

In [ ]:
len(vectorizer_desc.vocabulary_)

In [ ]:
train_cvec, validation_cvec = train_test_split(data, test_size=0.2, random_state=1000)

In [ ]:
description_train_cvec = vectorizer_desc.transform(train_cvec["item_description_clean"].values)

In [ ]:
description_validation_cvec = vectorizer_desc.transform(validation_cvec["item_description_clean"].values)

In [ ]:
vectorizer_name = CountVectorizer()
vectorizer_name.fit(data["name_clean"].values)

In [ ]:
len(vectorizer_name.vocabulary_)

In [ ]:
name_train_cvec = vectorizer_name.transform(train_cvec["name_clean"].values)
name_validation_cvec = vectorizer_name.transform(validation_cvec["name_clean"].values)

In [ ]:
categorical_train_cvec= train_cvec[["item_condition_id","shipping","category_name_l","brand_name_l"]]
print("Train:")
print(categorical_train_cvec.head())
print("\nValidation:")
categorical_validation_cvec= validation_cvec[["item_condition_id","shipping","category_name_l","brand_name_l"]]
print(categorical_validation_cvec.head())

In [ ]:
y_train_cvec=train_cvec["price"].values
y_validation_cvec=validation_cvec["price"].values

In [ ]:
def getModel_bow():
    inputA = Input(categorical_train_cvec.shape[1])
    inputName = Input(name_train_cvec.shape[1])
    inputDesc = Input(description_train_cvec.shape[1])
    concat = Concatenate()([inputName, inputDesc, inputA])

    x = Dropout(0.1)(concat)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(16, activation='relu')(x)
    
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[inputName ,inputDesc, inputA], outputs=x)

    return model

In [ ]:
model_cvec = getModel_bow()
model_cvec.summary()

In [ ]:
plot_model(model_cvec)

In [ ]:
model_cvec.compile(loss = root_mean_squared_logarithmic_error, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

In [ ]:
history_cvec = model_cvec.fit(x=[name_train_cvec, description_train_cvec, categorical_train_cvec.values],
                    y=y_train_cvec,
                    epochs=10,
                    verbose=True,
                    validation_data=(
                        [name_validation_cvec, description_validation_cvec, categorical_validation_cvec.values]
                        , y_validation_cvec),
                    batch_size=512)

BOW con pulizia:
loss: 0.4549 - mse: 877.1694 - mae: 10.7754 - root_mean_squared_error: 29.6144 - mean_squared_logarithmic_error: 0.2073 - root_mean_squared_logarithmic_error: 0.4549 - val_loss: 0.4572 - val_mse: 823.4496 - val_mae: 10.7306 - val_root_mean_squared_error: 28.6958 - val_mean_squared_logarithmic_error: 0.2095 - val_root_mean_squared_logarithmic_error: 0.4573

BOW senza pulizia words:
loss: 0.4537 - mse: 892.7444 - mae: 10.7434 - root_mean_squared_error: 29.8743 - mean_squared_logarithmic_error: 0.2063 - root_mean_squared_logarithmic_error: 0.4537 - val_loss: 0.4554 - val_mse: 848.4371 - val_mae: 10.7117 - val_root_mean_squared_error: 29.1279 - val_mean_squared_logarithmic_error: 0.2079 - val_root_mean_squared_logarithmic_error: 0.4555

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    rmsle = history.history['root_mean_squared_logarithmic_error']
    val_rmsle = history.history['val_root_mean_squared_logarithmic_error']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(rmsle) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, rmsle, 'b', label='Training rmsle')
    plt.plot(x, val_rmsle, 'r', label='Validation rmsle')
    plt.title('Training and validation root_mean_squared_logarithmic_error')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)

# Transformers

In [ ]:
pret_model_trans = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
pret_model_trans.trainable = False

tokenizer_trans = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
data = data[0:1000]

In [ ]:
train_trans, validation_trans = train_test_split(data, test_size=0.2, random_state=1000)

In [ ]:
y_train_trans = train_trans["price"]
y_train_trans

In [ ]:
y_validation_trans = validation_trans["price"]
y_validation_trans

todo: use cleaned? or let bert handle everything?

In [ ]:
inputA_train_trans = train_trans[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
inputA_validation_trans = validation_trans[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]].values.astype('int32')

In [ ]:
inputName_train_trans = tokenizer_trans(train_trans["name"].to_list(),
                                  return_tensors="tf",
                                  padding=True)
#inputName_trans

In [ ]:
inputName_validation_trans = tokenizer_trans(validation_trans["name"].to_list(),
                                  return_tensors="tf",
                                  padding=True)
#inputName_trans

In [ ]:
inputName_shape_trans = (inputName_train_trans['input_ids'].shape[1],
                         inputName_train_trans['attention_mask'].shape[1])

inputName_shape_trans

In [ ]:
inputDesc_train_trans = tokenizer_trans(train_trans["item_description"].to_list(),
                                  return_tensors="tf",
                                  padding=True)

In [ ]:
inputDesc_validation_trans = tokenizer_trans(validation_trans["item_description"].to_list(),
                                  return_tensors="tf",
                                  padding=True)

In [ ]:
inputDesc_shape_trans = (inputDesc_train_trans['input_ids'].shape[1],
                         inputDesc_train_trans['attention_mask'].shape[1])
inputDesc_shape_trans

In [ ]:
def getModel_trans():
    inputA = Input(shape=(4,))
    Ad = inputA    
    
    inputName_ids = Input(shape=(inputName_shape_trans[0],), dtype='int32')
    inputName_mask = Input(shape=(inputName_shape_trans[1],), dtype='int32')

    Np = pret_model_trans(inputName_ids, attention_mask=inputName_mask)[0]

    Nd = LSTM(12,return_sequences=True)(Np)
    Nd = Flatten()(Nd)

    inputDesc_ids = Input(shape=(inputDesc_shape_trans[0],), dtype='int32')
    inputDesc_mask = Input(shape=(inputDesc_shape_trans[1],), dtype='int32')

    Dp = pret_model_trans(inputDesc_ids, attention_mask=inputDesc_mask)[0]

    Dd = LSTM(16,return_sequences=True)(Dp)
    Dd = LSTM(8,return_sequences=True)(Dd)
    Dd = Flatten()(Dd)

    concat = Concatenate()([Ad, Nd, Dd])

    x = Dropout(0.1)(concat)    
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(16, activation='relu')(x)
    
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[
                          inputA,
                          inputName_ids,
                          inputName_mask,
                          inputDesc_ids,
                          inputDesc_mask
                          ], outputs=x)
    
    return model
    

In [ ]:
model_trans = getModel_trans()

In [ ]:
model_trans.summary()

In [ ]:
plot_model(model_trans)

In [ ]:
msle = tf.keras.losses.MeanSquaredLogarithmicError()

def root_mean_squared_logarithmic_error(y_true, y_pred):
    return K.sqrt(msle(y_true, y_pred))

In [ ]:
model_trans.compile(loss = root_mean_squared_logarithmic_error, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
history_trans = model_trans.fit(x=[inputA_train_trans,
                   inputName_train_trans['input_ids'],
                   inputName_train_trans['attention_mask'],
                   inputDesc_train_trans['input_ids'],
                   inputDesc_train_trans['attention_mask']
                  ],
                   y=y_train_trans,
                    epochs=2,
                    verbose=True,
                    validation_data=([
                                      inputA_validation_trans,
                                      inputName_validation_trans['input_ids'],
                                      inputName_validation_trans['attention_mask'],
                                      inputDesc_validation_trans['input_ids'],
                                      inputDesc_validation_trans['attention_mask'],
                                      ], 
                                     y_validation_trans),
                    batch_size=512)